In [70]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score


import tensorflow as tf
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import LSTM
from keras.layers import Dropout

In [71]:
result = pd.read_csv("trainingset.csv").drop("Unnamed: 0", axis=1)
result = result.astype(np.float32)

In [72]:
X = result.drop("Hourly Demand Met (in MW)", axis=1)
y = result["Hourly Demand Met (in MW)"]

In [78]:
train_X, test_X, train_y, test_y = train_test_split(X, y ,test_size=0.2)
train_X = train_X.values
print(train_X.shape)
train_X = train_X.reshape((train_X.shape[0], train_X.shape[1], 1))
print(train_X.shape)

train_y = train_y.values

test_X = test_X.values
test_X = test_X.reshape((test_X.shape[0], test_X.shape[1], 1))

test_y = test_y.values

(56083, 14)
(56083, 14, 1)


In [75]:
import datetime
import os

def create_tensorboard_callback():
    logdir = os.path.join("Logs",
    datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    
    return tf.keras.callbacks.TensorBoard(logdir)

In [76]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                                  patience=3)

In [80]:
tensorboard = create_tensorboard_callback()

model = Sequential()
model.add(LSTM(100, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(loss='mean_absolute_error', optimizer='adam')

model.fit(train_X, train_y, epochs=100, batch_size=32, validation_data=(test_X, test_y), verbose=2, shuffle=False, callbacks=[tensorboard, early_stopping])

Epoch 1/10


2024-08-31 20:22:39.924301: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-08-31 20:22:39.925332: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-08-31 20:22:39.925861: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1753/1753 - 10s - loss: 3592.1868 - val_loss: 3500.8750 - 10s/epoch - 6ms/step
Epoch 2/10
1753/1753 - 9s - loss: 3416.1982 - val_loss: 3325.7971 - 9s/epoch - 5ms/step
Epoch 3/10
1753/1753 - 8s - loss: 3242.5730 - val_loss: 3152.7183 - 8s/epoch - 5ms/step
Epoch 4/10
1753/1753 - 8s - loss: 3069.5374 - val_loss: 2979.8154 - 8s/epoch - 5ms/step
Epoch 5/10
1753/1753 - 8s - loss: 2895.5361 - val_loss: 2804.1641 - 8s/epoch - 5ms/step
Epoch 6/10
1753/1753 - 8s - loss: 2718.6541 - val_loss: 2626.6824 - 8s/epoch - 5ms/step
Epoch 7/10
1753/1753 - 8s - loss: 2542.2285 - val_loss: 2450.3391 - 8s/epoch - 5ms/step
Epoch 8/10
1753/1753 - 9s - loss: 2365.3445 - val_loss: 2274.5327 - 9s/epoch - 5ms/step
Epoch 9/10
1753/1753 - 8s - loss: 2193.4910 - val_loss: 2106.3530 - 8s/epoch - 5ms/step
Epoch 10/10
1753/1753 - 8s - loss: 2032.5897 - val_loss: 1952.6210 - 8s/epoch - 5ms/step


In [ ]:
%tensorboard --logdir /Logs

UsageError: Line magic function `%tensorboard` not found.


In [ ]:
y_preds = model.predict(test_X)

 31/384 [=>............................] - ETA: 0s  

2024-08-31 18:53:48.163711: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-08-31 18:53:48.164483: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-08-31 18:53:48.165041: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

384/384 [==============================] - 1s 2ms/step


In [ ]:
mean_absolute_error(y_preds, test_y)

551.6644

In [ ]:
y_preds

array([[2938.0857],
       [4529.811 ],
       [4709.7827],
       ...,
       [2689.0618],
       [4296.993 ],
       [2692.046 ]], dtype=float32)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional

model = Sequential()
model.add(Bidirectional(LSTM(100, return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2]))))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(1))

model.compile(loss='mean_absolute_error', optimizer='adam')

model.fit(train_X, train_y, epochs=150, batch_size=32, validation_data=(test_X, test_y), verbose=2, shuffle=False, callbacks=[tensorboard, early_stopping])

Epoch 1/150


2024-08-31 18:56:36.748030: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-08-31 18:56:36.748704: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-08-31 18:56:36.749322: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1534/1534 - 21s - loss: 3632.6260 - val_loss: 3643.6790 - 21s/epoch - 14ms/step
Epoch 2/150
1534/1534 - 22s - loss: 3554.2117 - val_loss: 3565.6367 - 22s/epoch - 14ms/step
Epoch 3/150
1534/1534 - 22s - loss: 3476.1809 - val_loss: 3487.6685 - 22s/epoch - 14ms/step
Epoch 4/150
1534/1534 - 20s - loss: 3398.0935 - val_loss: 3409.7070 - 20s/epoch - 13ms/step
Epoch 5/150
1534/1534 - 18s - loss: 3320.2246 - val_loss: 3331.7820 - 18s/epoch - 12ms/step
Epoch 6/150
1534/1534 - 22s - loss: 3242.5303 - val_loss: 3253.8787 - 22s/epoch - 14ms/step
Epoch 7/150
1534/1534 - 18s - loss: 3164.4653 - val_loss: 3175.9351 - 18s/epoch - 12ms/step
Epoch 8/150
1534/1534 - 19s - loss: 3086.0605 - val_loss: 3097.9866 - 19s/epoch - 13ms/step
Epoch 9/150
1534/1534 - 20s - loss: 3008.5422 - val_loss: 3020.0964 - 20s/epoch - 13ms/step
Epoch 10/150
1534/1534 - 22s - loss: 2930.9553 - val_loss: 2942.2131 - 22s/epoch - 14ms/step
Epoch 11/150
1534/1534 - 20s - loss: 2852.8494 - val_loss: 2864.3064 - 20s/epoch - 13ms/ste

In [ ]:
y_preds = model.predict(test_X)
mean_absolute_error(y_preds, test_y)

2024-08-31 19:29:03.920843: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-08-31 19:29:03.921713: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-08-31 19:29:03.922312: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

384/384 [==============================] - 2s 4ms/step


544.2008

In [ ]:
r2_score(y_preds, test_y)

0.50846266746521